In [26]:
from interpreting_neurons_utils import *

- Look at python fuzzy logic libraries
- Experiment 1
    - Make the general setup, works for
        - different kinds of membership functions of output neurons
        - different kind of cost functions
        - different kind of input (Input are the fuzzy variables)
    - Define single fuzzy membership functions for all neurons
        - Look at histogram and do shit
        - Just do Negative, Positive
    - Use softmax for fuzzy input variables
    - use L1 sparcity constraint, extra for > 1
- ⇒
    - How good is MSE of neuron activations
    - Histogram of the weights
    - How realistic are the rules learned (are they the same that I would come up with)

- I think I need to do all the Fuzzy Logic shit by hand ...
- I think it makes sense to initialize all the fuzzy weights near 0.5

- First: not general?!
- Training Loop
    - Residual Stream
    - Proberesults (Softmax)
    - Fuzzy Inference -> Neuron Activation Pred (This is the step where things are learned)
    - Loss
    - Update

In [27]:
from shapely.geometry import Polygon
from shapely.ops import unary_union
import pickle
import wandb

In [28]:
class MemberShipFunc:
    def __init__(self, a : float, b : float, c : float, d : float, height = 1):
        self.a = a
        self.b = b
        self.c = c
        self.d = d
        self.height = height
        self.trapezoid = Polygon([(a, 0), (b, height), (c, height), (d, 0)])
    
    def __call__(self, x):
        if x <= self.a or x >= self.d:
            return 0
        elif x < self.b:
            return (x - self.a) / (self.b - self.a) * self.height
        elif x < self.c:
            return self.height
        else:
            return (self.d - x) / (self.d - self.c) * self.height
        
    def __str__(self):
        return f"MemberShipFunc({self.a}, {self.b}, {self.c}, {self.d})"
    
    def __mul__(self, scalar : Float[Tensor, ""] | float):
        if type(scalar) == t.Tensor:
            scalar = scalar.item()
        return MemberShipFunc(
            self.a,
            self.b,
            self.c,
            self.d,
            height = scalar,
        )
    
def union(member_ship_funcs : List[MemberShipFunc]):
    return unary_union([member_ship_func.trapezoid for member_ship_func in member_ship_funcs])

def defuzz(member_ship_funcs : List[MemberShipFunc]):
    unioned = union(member_ship_funcs)
    return unioned.centroid.x

In [29]:
debug = False

In [30]:
def get_centroid(trapezoid_metadata : List[Tuple[float, float, float, float]], trapezoid_heights : Float[Tensor, "tropezoid batch pos neurons"]):
    # This is the Center of Sums Method. Theres also the center of Gravity Method
    # TODO: get rid of the for loop, vectorize this
    trapezoids, batches, positions, neurons = trapezoid_heights.shape
    member_ship_funcs = []
    numerrator = t.zeros((batches, positions, neurons)).to(device)
    denomenator = t.zeros((batches, positions, neurons)).to(device)
    for i, (a, b, c, d) in enumerate(trapezoid_metadata):
        trapezoid_height = trapezoid_heights[i]
        b_new = a + trapezoid_height * (b-a)
        c_new = d - trapezoid_height * (d-c)
        long_base = d - a
        short_base = c_new - b_new 
        individual_centroid = ((long_base + 2 * short_base) * trapezoid_height) / (3*(long_base + short_base))
        individual_area = (long_base + short_base) / 2 * trapezoid_height
        numerrator += individual_centroid * individual_area
        denomenator += individual_area
    return numerrator / denomenator

In [31]:
import collections, gc, resource, torch
def debug_memory():
    print('maxrss = {}'.format(
        resource.getrusage(resource.RUSAGE_SELF).ru_maxrss))
    tensors = collections.Counter(
        (str(o.device), str(o.dtype), tuple(o.shape))
        for o in gc.get_objects()
        if torch.is_tensor(o)
    )
    for line in sorted(tensors.items()):
        print('{}\t{}'.format(*line))

In [32]:
plot_game(focus_games_string, 13, end_move=16)

In [33]:
"""layer = 1
module = f"blocks.{layer}.ln2.hook_normalized"
neuron = 426
neurons_count = 2048
variables_count = 64 * (3 + 2 + 2)
rules_count = 10

neuron_negative_func = MemberShipFunc(-5, -3, 0, 0.5)
neuron_positive_func = MemberShipFunc(0, 0.5, 3, 5)"""

neuron_membership_funcs = [(-5, -3, 0, 0.5), (0, 0.5, 3, 5)]

# TODO: First make funcitons narrow, then gerenerell
def variables_to_tensor(input_variables : Dict[str, Tuple]) -> Float[Tensor, "variables"]:
    out = t.Tensor((64 * (3 + 2 + 2)))
    i = 0
    for _, value in input_variables.items():
        for j in range(len(value)):
            out[i] = value[j]
            i += 1
    return out

def tensor_to_variables(input_tensor : Float[Tensor, "variables"]) -> Dict[str, Tuple]:
    pass

def compute_fuzzy_out(fuzzy_weights, input_variables, pos, neurons, rules):
    fuzzy_weights_r = einops.repeat(fuzzy_weights, "neurons rules variables -> pos neurons rules variables", pos=pos)
    input_variables_r = einops.repeat(input_variables, "pos variables -> pos neurons rules variables", neurons=neurons, rules=rules)
    updated_variables : Float[Tensor, "pos neuron rules variables"] = 1 - fuzzy_weights_r * (1 - input_variables_r)
    # fuzzy_mid = einops.reduce(updated_variables, "pos neurons rules variables -> pos neurons rules", "min")  # AND Gate
    # fuzzy_out = einops.reduce(fuzzy_mid, "pos neurons rules -> pos neurons", "max")  # OR Gate
    updated_variables = updated_variables.min(dim=-1).values  # AND Gate
    updated_variables = updated_variables.max(dim=-1).values  # OR Gate
    return updated_variables

def compute_fuzzy_mid(fuzzy_weights, input_variables, v, batch, pos, neurons, rules):
    return 1 - einops.repeat(fuzzy_weights[:, :, v], "neurons rules -> batch pos neurons rules", batch=batch, pos=pos) * (1 - einops.repeat(input_variables[:, :, v], "batch pos -> batch pos neurons rules", neurons=neurons, rules=rules))

def compute_fuzzy_out_new(fuzzy_weights, input_variables):
    neurons, rules, variables = fuzzy_weights.shape
    batch, pos, variables = input_variables.shape
    # fuzzy_weights_r = einops.repeat(fuzzy_weights, "neurons rules variables -> pos neurons rules variables")
    # input_variables_r = einops.repeat(input_variables, "batch pos variables -> pos neurons rules variables")
    fuzzy_mid = t.zeros((batch, pos, neurons, rules)).to(device)
    for v in range(variables):
        if debug:
            print(f"Fuzzy Logic started for variable {v}")
            debug_memory()
        fuzzy_mid = t.min(fuzzy_mid, compute_fuzzy_mid(fuzzy_weights, input_variables, v, batch, pos, neurons, rules))
    # fuzzy_mid = einops.reduce(updated_variables, "pos neurons rules variables -> pos neurons rules", "min")  # AND Gate
    # fuzzy_out = einops.reduce(fuzzy_mid, "pos neurons rules -> pos neurons", "max")  # OR Gate
    updated_variables = updated_variables.min(dim=-1).values  # AND Gate
    updated_variables = updated_variables.max(dim=-1).values  # OR Gate
    return updated_variables


def perform_fuzzy_logic(input_variables : Float[Tensor, "batch pos variables"], fuzzy_weights : Float[Tensor, "neuron rules variables"]) -> Dict[str, Tuple]:
    batch, pos, variables = input_variables.shape
    neurons, rules, _ = fuzzy_weights.shape
    if neurons == 1:
        fuzzy_weights_r = einops.repeat(fuzzy_weights, "neurons rules variables -> batch pos neurons rules variables", batch=batch, pos=pos)
        input_variables_r = einops.repeat(input_variables, "batch pos variables -> batch pos neurons rules variables", neurons=neurons, rules=rules)
        updated_variables : Float[Tensor, "batch pos neuron rules variables"] = 1 - fuzzy_weights_r * (1 - input_variables_r)
        fuzzy_mid = einops.reduce(updated_variables, "batch pos neurons rules variables -> batch pos neurons rules", "min")  # AND Gate
        fuzzy_out = einops.reduce(fuzzy_mid, "batch pos neurons rules -> batch pos neurons", "max")  # OR Gate
        return fuzzy_out
    fuzzy_out_complete = []
    for b in range(batch):
        if debug:
            print(f"Fuzzy Logic started for batch {b}")
            debug_memory()
        fuzzy_out = compute_fuzzy_out(fuzzy_weights, input_variables[b], pos, neurons, rules)
        fuzzy_out_complete.append(fuzzy_out)
    fuzzy_out_complete = t.stack(fuzzy_out_complete, dim=0)
    # fuzzy_out_complete = compute_fuzzy_out_new(fuzzy_weights, input_variables)
    return fuzzy_out_complete

def perform_defuzzification(neuron_positive : Float[Tensor, "batch pos neurons"]) -> Float[Tensor, "batch pos neurons"]:
    # I need the fuzzy membership funcitions for the neurons and some algorithm
    if debug:
        print("Defuzzification started")
    neuron_negative = 1 - neuron_positive
    batch, pos, neurons = neuron_positive.shape
    neuron_activation = get_centroid(neuron_membership_funcs, t.stack([neuron_negative, neuron_positive], dim=0))
    if debug:
        print("Defuzzification ended")
    return neuron_activation

def full_fuzzy_inference(input_variables : Float[Tensor, "batch pos variables"], fuzzy_weights : Float[Tensor, "neuron rules variables"]) -> Dict[str, float]: # Probe Results -> Neuron Activations
    """
    Perform fuzzy inference on the input variables.
    """
    # input_variables = input_variables_to_tensor(input_variables)
    # 1. Turn Fuzzy Variable into another Fuzzy Variable using the learned logical rules
    # The shape here will change with better neuron membership functions
    fuzzy_out : Float[Tensor, "batch pos neurons"] = perform_fuzzy_logic(input_variables, fuzzy_weights)
    # 2. Turn the Fuzzy Variable into a crisp value using defuzzification
    neuron_activations : Float[Tensor, "batch pos neurons"] = perform_defuzzification(fuzzy_out)
    return neuron_activations

def get_variables(resid_mid : Float[Tensor, "batch pos d_model"], layer) -> Float[Tensor, "batch pos variables"]:
    probe_results_all = []
    for probe_name in ["linear", "flipped", "placed"]:
        probe = get_probe(layer, probe_name, "mid")[0]
        probe_result = einops.einsum(resid_mid, probe, "batch pos d_model, d_model rows cols options -> batch pos rows cols options").softmax(dim=-1)
        probe_result = einops.rearrange(probe_result, "batch pos rows cols options -> batch pos (rows cols options)")
        probe_results_all.append(probe_result)
    probe_results_all = t.cat(probe_results_all, dim=-1)
    return probe_results_all

def init_weights(neurons_count, rules_count, variables_count, manual=False) -> Float[Tensor, "neurons rules variables"]:
    if manual:
        fuzzy_weights = t.zeros(size=(neurons_count, rules_count, 8, 8, 7), requires_grad=False).to(device)
        fuzzy_weights[0, 0, *label_to_tuple("E2"), 5] = 1
        fuzzy_weights[0, 0, *label_to_tuple("E3"), 3] = 1
        fuzzy_weights[0, 0, *label_to_tuple("F2"), 3] = 1

        fuzzy_weights[0, 1, *label_to_tuple("F3"), 5] = 1
        fuzzy_weights[0, 1, *label_to_tuple("E3"), 3] = 1
        fuzzy_weights[0, 1, *label_to_tuple("F2"), 3] = 1

        fuzzy_weights[0, 2, *label_to_tuple("C5"), 5] = 1
        fuzzy_weights[0, 2, *label_to_tuple("E3"), 3] = 1
        fuzzy_weights[0, 2, *label_to_tuple("F2"), 3] = 1

        fuzzy_weights[0, 3, *label_to_tuple("B6"), 5] = 1
        fuzzy_weights[0, 3, *label_to_tuple("E3"), 3] = 1
        fuzzy_weights[0, 3, *label_to_tuple("F2"), 3] = 1

        fuzzy_weights[0, 4, *label_to_tuple("F5"), 5] = 1
        fuzzy_weights[0, 4, *label_to_tuple("E4"), YOURS] = 1
        fuzzy_weights[0, 4, *label_to_tuple("F3"), YOURS] = 1
        fuzzy_weights[0, 4, *label_to_tuple("E2"), YOURS] = 1
        fuzzy_weights[0, 4, *label_to_tuple("F1"), YOURS] = 1

        fuzzy_weights = einops.rearrange(fuzzy_weights, "neurons rules rows cols option_probes -> neurons rules (rows cols option_probes)")
        fuzzy_weights.requires_grad = True
        return fuzzy_weights
    
    fuzzy_weights = t.randn(size=(neurons_count, rules_count, variables_count), requires_grad=False).to(device)
    fuzzy_weights = fuzzy_weights.normal_(0.3, 0.01)
    fuzzy_weights.requires_grad = True
    return fuzzy_weights

# Try one inference
"""variables = get_variables(focus_cache[module])
variables = variables[:2, :]
weights = init_weights()
neuron_activations = full_fuzzy_inference(variables, weights)
print(neuron_activations[0, 0, neuron])"""

'variables = get_variables(focus_cache[module])\nvariables = variables[:2, :]\nweights = init_weights()\nneuron_activations = full_fuzzy_inference(variables, weights)\nprint(neuron_activations[0, 0, neuron])'

In [34]:
@dataclass
class FuzzyTrainingArgs():
    # Which layer, and which positions in a game sequence to probe
    def __init__(self, run_number : int, layer : int, single_neuron=None):
        self.debug = False
        self.run_number = run_number
        self.layer: int = layer
        self.pos_start: int = 0
        self.pos_end: int = 59
        self.length: int = self.pos_end - self.pos_start
        self.max_epochs = 1
        self.single_neuron = single_neuron
        self.rules_count = 10
        self.neurons_count = 2048 if single_neuron is None else 1
        self.variables_count = 64 * (3 + 2 + 2)
        self.manual = False

        self.num_games_train: int = 250
        self.batch_size: int = 2

        self.lr: float = 1e-2
        self.betas: Tuple[float, float] = (0.9, 0.99)
        self.wd: float = 0.1

        self.run_name: str = f"Fuzzy_{self.run_number}"
        self.full_run_name: str = f"Fuzzy_L{self.layer}_{self.run_number}"

        # wnadb
        self.wandb_project: str = f"Train Fuzzy Logic Gates Test"

In [35]:
class FuzzyTrainer:
    def __init__(self, model: HookedTransformer, args: FuzzyTrainingArgs, train_base_resid):
        self.model = model
        self.args = args
        self.probe_names = {
            "linear" : ["empty", "yours", "mine"],
            "flipped" : ["flipped", "not_flipped"],
            "placed" : ["placed", "not_placed"],
        }
        self.weights : Float[Tensor, "neurons rules variables"] = init_weights(self.args.neurons_count, self.args.rules_count, self.args.variables_count, self.args.manual)

    def training_step(self, indices: Int[Tensor, "game_idx"], train_or_val="train") -> t.Tensor:
        # Get the game sequences and convert them to state stacks
        games_int = board_seqs_int[indices.cpu()]
        games_str = board_seqs_string[indices.cpu()]
        batch_size = self.args.batch_size
        game_len = self.args.length
        # options = self.args.options
        d_model = 512

        # games_int = tensor of game sequences, each of length 60
        # This is the input to our model
        assert isinstance(games_int, Int[Tensor, f"batch={batch_size} full_game_len=60"])

        # SOLUTION
        with t.inference_mode():
            _, cache = model.run_with_cache(
                games_int[:, :-1].to(device),
                return_type=None,
                names_filter=lambda name: name == f"blocks.{self.args.layer}.ln2.hook_normalized" or name == f"blocks.{layer}.mlp.hook_post"
            )
            resid_layer_norm = cache[f"blocks.{self.args.layer}.ln2.hook_normalized"][:, self.args.pos_start: self.args.pos_end]
            neuron_activations : Float[Tensor, "batch pos neurons"] = cache["mlp_post", layer][:, self.args.pos_start: self.args.pos_end]
            if self.args.single_neuron is not None:
                neuron_activations = neuron_activations[:, :, self.args.single_neuron].unsqueeze(-1)
            # resid = cache["resid_mid", layer][:, self.args.pos_start: self.args.pos_end]
        resid_layer_norm = resid_layer_norm.clone().detach().to(device)
        variables : Float[Tensor, "batch pos variables"] = get_variables(resid_layer_norm, self.args.layer)
        neuron_activations_pred : Float[Tensor, "batch pos neurons"] = full_fuzzy_inference(variables, self.weights)

        mse : Float[Tensor, "batch pos neurons"] = (neuron_activations - neuron_activations_pred)**2
        mse_loss = mse.mean()

        # TODO: Create a fancy loss function that everything at ones (Forces the weights to be either 0 or 1, but allowing transitions between these two)
        # This is currently a placeholder
        sparcity_loss = self.weights.abs().mean()
        negative_loss = self.weights.clamp(max=0).mean()
        over_one_loss = self.weights.clamp(min=1).mean()
        loss = mse_loss + self.args.wd * (sparcity_loss + negative_loss + over_one_loss)
        
        if train_or_val == "train" and self.step % 10 == 0 and not self.args.debug:
            wandb.log({f"{train_or_val}_loss": loss.item()}, step=self.step)
        self.step += 1

        return loss
        
    def save_weights(self, object, path):
        folder_path = "/".join(path.split("/")[:-1])
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        with open(path, "wb") as f:
            pickle.dump(object, f)

    def shuffle_indices(self, start, num_games):
        '''
        Helper function
        '''
        n_indices = num_games - (num_games % self.args.batch_size)
        full_train_indices = t.randperm(num_games)[:n_indices] + start
        full_train_indices = einops.rearrange(full_train_indices, "(batch_idx game_idx) -> batch_idx game_idx", game_idx=self.args.batch_size)
        return full_train_indices
    
    def shuffle_training_indices(self):
        '''
        Returns the tensors you'll use to index into the training data.
        '''
        return self.shuffle_indices(0, self.args.num_games_train)

    def train(self):
        print(f"Training Probe: {self.args.full_run_name}")
        self.step = 0
        if not self.args.debug:
            wandb.login()
            wandb.init(project=self.args.wandb_project, name=self.args.full_run_name, config=self.args)

        optimizer = t.optim.AdamW([self.weights], lr=self.args.lr, betas=self.args.betas)

        for epoch in range(self.args.max_epochs):
            full_train_indices = self.shuffle_training_indices()
            progress_bar_train = tqdm(full_train_indices)
            for indices in progress_bar_train:
                loss = self.training_step(indices)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                progress_bar_train.set_description(f"Train_Loss = {loss:.4f}")

        if not self.args.debug:
            wandb.finish()
            self.save_weights(self.weights, f"fuzzy_weights/_{self.args.full_run_name}.pth")
        print("Probe Trained and Saved")

In [36]:
# set torch to random seed and print the seed
t.manual_seed(42)
run_number = 0

args = FuzzyTrainingArgs(run_number, layer=0, single_neuron=368)
args.num_games_train = 128
args.batch_size = 128
args.rules_count = 5
args.manual = True
args.debug = True
trainer = FuzzyTrainer(model, args, train_base_resid=False)
trainer.train()

Training Probe: Fuzzy_L0_0


  0%|          | 0/1 [00:00<?, ?it/s]

Probe Trained and Saved


In [37]:
def print_rules(weights : Float[Tensor, "neurons rules variables"], neuron : int = 0):
    thresh = 0.1
    # TODO: Why did I put all these dimensions together in the first place? I guess it is easier to understand this way
    variables = einops.rearrange(weights, "neurons rules (rows cols options_probes) -> neurons rules rows cols options_probes", rows=8, cols=8, options_probes=7)[neuron]
    rules, rows, cols, options_probes = variables.shape
    for rule in range(rules):
        for row in range(rows):
            for col in range(cols):
                for options_probe in range(options_probes):
                    if options_probe < 3:
                        option = options_probe
                        probe = 0
                    elif options_probe < 5:
                        option = options_probe - 3
                        probe = 1
                    else:
                        option = options_probe - 5
                        probe = 2
                    weight = variables[rule, row, col, options_probe]
                    probe_str = ["linear", "flipped", "placed"][probe]
                    option_str = get_direction_str(probe_str, option)
                    label = tuple_to_label((row, col))
                    if weight > thresh:
                        print(f"Rule {rule}, {label} is {option_str}: {weight}")
print_rules(trainer.weights, neuron=0)

Rule 0, E2 is placed: 0.9899004697799683
Rule 0, E3 is flipped: 0.9898999929428101
Rule 0, F2 is flipped: 0.9899000525474548
Rule 1, E3 is flipped: 0.9898999929428101
Rule 1, F2 is flipped: 0.9899011254310608
Rule 1, F3 is placed: 0.9898999929428101
Rule 2, C5 is placed: 0.9898999929428101
Rule 2, E3 is flipped: 0.9898999929428101
Rule 2, F2 is flipped: 0.9899002313613892
Rule 3, B6 is placed: 0.9898999929428101
Rule 3, E3 is flipped: 0.9898999929428101
Rule 3, F2 is flipped: 0.9899001717567444
Rule 4, E2 is yours: 0.9899011254310608
Rule 4, E4 is yours: 0.9899011254310608
Rule 4, F1 is yours: 0.9898999929428101
Rule 4, F3 is yours: 0.9898999929428101
Rule 4, F5 is placed: 0.9898999929428101


In [38]:
print(trainer.weights.shape)
for i in range(10):
    print((trainer.weights[:, i, :] > 0.1).sum())
    print((trainer.weights[:, i, :] > 0.3).sum())

torch.Size([1, 5, 448])
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')
tensor(5, device='cuda:0')
tensor(5, device='cuda:0')


IndexError: index 5 is out of bounds for dimension 1 with size 5